In [1]:
from read_json import read_json # function to process data
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [2]:
# hehe = read_json('yelp_academic_dataset_checkin.json')

In [3]:
# hehe.head()

In [4]:
df_business = read_json('yelp_academic_dataset_business.json')
df_review = read_json('yelp_academic_dataset_review.json')

In [5]:
df_review.shape

(6990280, 9)

In [6]:
# months = {0:'Janurary',1:'February',2:'March',3:'April',4:'May',5:'June',
#          6:'July',7:'August',8:'September',9:'October',10:'November',11:'December'}

In [7]:
review = df_review.rename(columns={"stars": "ratings"})
review['date'] = pd.to_datetime(review['date'])

In [8]:
review = review[(review['date'] > '2019-01-01') & (review['date'] < '2019-12-31')]

In [9]:
review.shape

(904684, 9)

In [10]:
  # process date

review['year'] = review['date'].dt.year
review['month'] = review['date'].dt.month

# transform to day of the week
review['day_of_week'] = review['date'].dt.dayofweek
review['season'] = np.where(review['month'] <= 2, 'Spring',
                            np.where(review['month'] <= 5, 'Summer',
                                     np.where(review['month'] <= 8, 'Automne', 'Winter')))
#review['month'] = review['month'].map(months)
review['is_weekend'] = np.where(review['day_of_week'] == 0, 'Monday',
                                np.where(review['day_of_week'] == 1, 'Tuseday',
                                         np.where(review['day_of_week'] == 2, 'Wedseday',
                                                  np.where(review['day_of_week'] == 3, 'Thursday',
                                                           np.where(review['day_of_week'] == 4, 'Friday',
                                                                    np.where(review['day_of_week'] == 5, 'Saturday', 'Sunday'))))))
review['is_weekend'] = np.where(
    review['day_of_week'] >= 5, 'Weekend', 'Weekday')
review['time_of_day'] = review['date'].dt.hour  # time of the day
review['time_of_day'] = np.where(review['time_of_day'] > 21, 'Night', np.where(review['time_of_day'] > 18, 'Evening', np.where(
    review['time_of_day'] > 14, 'Afternoon', np.where(review['time_of_day'] > 11, 'Noon', np.where(review['time_of_day'] > 8, 'Morning', 'Early_morning')))))

In [11]:
review.columns

Index(['review_id', 'user_id', 'business_id', 'ratings', 'useful', 'funny',
       'cool', 'text', 'date', 'year', 'month', 'day_of_week', 'season',
       'is_weekend', 'time_of_day'],
      dtype='object')

In [12]:
a = review['user_id'].value_counts()[review['user_id'].value_counts() >= 10]
review = review[review['user_id'].isin(list(a.index))]

In [13]:
df_business.shape,review.shape

((150346, 14), (207017, 15))

In [14]:
df = pd.merge(review, df_business)

In [15]:
df.columns

Index(['review_id', 'user_id', 'business_id', 'ratings', 'useful', 'funny',
       'cool', 'text', 'date', 'year', 'month', 'day_of_week', 'season',
       'is_weekend', 'time_of_day', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [16]:
del df['review_id']
del df['useful']
del df['funny']
del df['cool']
del df['postal_code']
del df['latitude']
del df['longitude']

In [17]:
del df['name']
del df['address']
#del df['city']
del df['review_count']
del df['is_open']
del df['attributes']
del df['categories']
del df['hours']

In [18]:
df.columns

Index(['user_id', 'business_id', 'ratings', 'text', 'date', 'year', 'month',
       'day_of_week', 'season', 'is_weekend', 'time_of_day', 'city', 'state',
       'stars'],
      dtype='object')

In [19]:
df['business_id'].value_counts()

vUrTGX_7HxqeoQ_6QCVz6g    176
ytynqOUb3hjKeJfRj5Tshw    140
WQ1oZ11HbYL7K7kj62ttAw    122
_78Lam9Jd0W6ZmnHTvdjMg    121
erBjfgY05PFI9UElgguoQQ    120
                         ... 
QjxBDMyURV2NsvzqCu8Zqw      1
VMWq7Dr6wC5xPkL3rEuQSw      1
yTxUljktDCBe8UHcZTyxfQ      1
F47i9gBPpEjxZqPNLr5oiw      1
klpmioRcIM0CkQokBKp9zQ      1
Name: business_id, Length: 52298, dtype: int64

In [ ]:
#del df['state']
del df['stars']

In [22]:
df_usable = df.copy()

In [23]:
df_usable['text'][0]

"Best cheesesteak in the city hands down. I say this as someone who tried the meat option before I became a vegetarian and as someone who has tried their vegan friendly version. Both are delicious. Their black bean burger is one of my favorites in Philadelphia. Their egg cream is delicious. Shakes are delicious. Fries are delicious. I haven't been in too long and need to make a trip back."

In [24]:
df_usable['Alone_Companion'] = np.where(df_usable['text'].str.contains('we|We'),'Companion','Alone')

In [25]:
del df_usable['text']
del df_usable['date']

In [26]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [27]:
del df_business['name']
del df_business['address']
del df_business['postal_code']
del df_business['latitude']
del df_business['longitude']
del df_business['hours']
del df_business['city']
del df_business['state']

In [28]:
df_business = df_business[df_business['business_id'].isin(list(set(df_usable['business_id'])))]

In [29]:
df_business['categories'].isna().sum()

11

In [30]:
del df_usable['year']

In [31]:
del df_business['is_open']
del df_business['attributes']
#del df_business['categories']

In [32]:
# df_business.to_csv('df_business.csv',index=False)
# df_usable.to_csv('df_usable.csv',index = False)
# df.to_csv('df.csv',index = False)

In [33]:
df_business.head()

,business_id,stars,review_count,categories
1,mpf3x-BjTdTEA3yCZrAYPw,3.0,15,"Shipping Centers, Local Services, Notaries, Ma..."
3,MTSW4McQd7CbVtyjqoe9mw,4.0,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,"Brewpubs, Breweries, Food"
6,n_0UpQx1hsNbnPUSlodU8w,2.5,13,"Sporting Goods, Fashion, Shoe Stores, Shopping..."
10,UJsufbvfyfONHeWdvAHKjA,3.5,6,"Department Stores, Shopping, Fashion"


In [34]:
def find_categories1(x):
    if x != None:
        return x.split(',')[0]
    else:
        return 'Unknown'
    
# def find_categories2(x):
#     if x != None:
#         if len(x.split(',')) == 2:
#             return x.split(',')[1]
#         else:
#             return 'Unknown'
#     else:
#         return 'Unknown'

# def find_categories3(x):
#     if x != None:
#         if len(x.split(',')) == 3:
#             return x.split(',')[2]
#         else:
#             return 'Unknown'
#     else:
#         return 'Unknown'

In [35]:
df_business['att1'] = df_business['categories'].map(find_categories1)
# df_business['att2'] = df_business['categories'].map(find_categories2)
# df_business['att3'] = df_business['categories'].map(find_categories3)

In [36]:
del df_business['categories']

In [37]:
# del df_business['is_open']
# del df_business['attributes']
# del df_business['categories']

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
b = le.fit_transform(df_business['business_id'])
i = df_business['business_id'].values
dict_co = dict(zip(i, b))
df_usable['business_id'] = df_usable['business_id'].map(dict_co)
df_business['business_id'] = b

In [39]:
df_business['att1'].value_counts()

Restaurants             7150
Food                    3084
Shopping                1482
Nightlife               1185
Beauty & Spas           1166
                        ... 
Web Design                 1
Osteopaths                 1
Country Dance Halls        1
Georgian                   1
Childbirth Education       1
Name: att1, Length: 935, dtype: int64

In [40]:
b = le.fit_transform(df_usable['user_id'])
df_usable['user_id'] = b

In [41]:
df_usable = df_usable.rename(columns = {'user_id':'user','business_id':'item','ratings':'cnt'})
#del df_usable['text']

In [42]:
def dataframe_trans(df_context, k):
    x = 0
    for col in df_context.columns[k:]:
        num = len(df_context[col].value_counts())
        le = LabelEncoder()
        y = le.fit_transform(df_context[col])
        df_context[col] = y
        df_context[col] = df_context[col] + x
        x += num
    return df_context

In [43]:
#df_usable['year'] = df_usable['year'].astype('object')
df_usable['day_of_week'] = df_usable['day_of_week'].astype('object')
df_usable['month'] = df_usable['month'].astype('object')

In [44]:
df_usable.dtypes

user                 int64
item                 int64
cnt                float64
month               object
day_of_week         object
season              object
is_weekend          object
time_of_day         object
city                object
state               object
Alone_Companion     object
dtype: object

In [45]:
# num = len(df_usable['month'].value_counts())
# le = LabelEncoder()
# y = le.fit_transform(df_usable['month'])
# #df_context[col] = y
# #df_context[col] = df_context[col] + x

In [46]:
df_usable = dataframe_trans(df_usable,3)

In [47]:
df_business['stars'] = np.where(df_business['stars'] > 4.5, 'Five_star', np.where(df_business['stars'] >=4, 'Four_star',np.where(df_business['stars'] >= 3, 'Three_star',np.where(df_business['stars'] >= 2, 'Two_star','One_star'))))

In [48]:
df_business['review_count'].describe()

count    52298.000000
mean        92.991032
std        192.428720
min          5.000000
25%         16.000000
50%         39.000000
75%         97.000000
max       7568.000000
Name: review_count, dtype: float64

In [49]:
df_business['review_count'] = np.where(df_business['review_count'] > 97, 'High_review',np.where(df_business['review_count'] > 16, 'Mid_review','Few_review'))

In [50]:
df_business = dataframe_trans(df_business,1)

In [51]:
df_business.to_csv('df_business.csv',index = False)

In [52]:
df_usable.to_csv('df_usable.csv',index=False)

In [55]:
df_train = df_usable.sample(frac = 0.8, random_state = 0, axis = 0)
df_rest = df_usable[~df_usable.index.isin(df_train.index)]
df_test = df_rest.sample(frac = 0.5, random_state = 0,axis = 0)
df_validation = df_rest[~df_rest.index.isin(df_test.index)]